# 감성분석

## Contents

- 1. VADER
- 2. WordNet

In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('/Users/jin/Documents/pythonstudy/NLP/frip_data_lda/lda_all.csv', index_col=0)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10740 entries, 0 to 10739
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   날짜             10740 non-null  object 
 1   내용             10740 non-null  object 
 2   별점             10740 non-null  float64
 3   제목             10740 non-null  object 
 4   카테고리           10740 non-null  object 
 5   지역             10740 non-null  object 
 6   가격             10740 non-null  int64  
 7   review         10740 non-null  object 
 8   s_review       10740 non-null  object 
 9   번역             10740 non-null  object 
 10  t_preprocess   10740 non-null  object 
 11  clean_word     10740 non-null  object 
 12  Drop_stopword  10740 non-null  object 
 13  Topic          10740 non-null  int64  
 14  Topic_code     10740 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 1.3+ MB


# VADER

In [11]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [12]:
senti_analyzer = SentimentIntensityAnalyzer()

In [30]:
data['vader'] = data['번역'].apply(senti_analyzer.polarity_scores)

In [36]:
data['v_neg'] = [vader['neg'] for vader in data['vader']]
data['v_pos'] = [vader['pos'] for vader in data['vader']]
data['v_neu'] = [vader['neu'] for vader in data['vader']]
data['v_com'] = [vader['compound'] for vader in data['vader']]

# Wordnet

In [40]:
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag

In [41]:
def penn_to_wn(tag):
    if tag.startswith('J'):  # 문자열.statswith('문자1') : 문자열 앞이 문자1로 시작하나?
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB

In [68]:
def swn_polarity(text):
    pos_sentiment= 0.0
    neg_sentiment= 0.0
    tokens_count = 0 
    pos_count = 0
    neg_count = 0
    lemmatizer = WordNetLemmatizer()
    raw_sentences = sent_tokenize(text)
    
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))
        for word, tag in tagged_sentence:
            
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV, wn.VERB):
                continue  # 명사, 형용사, 부사가 아니면 아무것도 안한다.
            
            # 그렇지 않으면, 원형화 한다. (명사, 형용사, 부사면 원형화 한다.)
            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma :  # 공백이면, 
                continue 
            # 원형으로 된 단어, tagging
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets : # 공백이면,
                continue
            
            synset = synsets[0] # 첫번째, 뜻으로 
            swn_synset = swn.senti_synset(synset.name()) # name() 태깅된 단어
            #sentiment += (swn_synset.pos_score() - swn_synset.neg_score())
            
            pos = swn_synset.pos_score()
            neg = swn_synset.neg_score()
            
            pos_sentiment += swn_synset.pos_score()
            neg_sentiment += swn_synset.neg_score()
            
            if pos > neg :
                pos_count += 1
            elif pos == neg:
                pass
            else : 
                neg_count += 1
            
            tokens_count += 1
    
    if not tokens_count :   # tokens_count 가 0이면
        return 0
    
    return {'pos_sent':pos_sentiment, 'neg_sent':neg_sentiment, 'pos_count':pos_count, 'neg_count':neg_count}

In [102]:
data['wordnet'] = data['번역'].apply(swn_polarity)

In [104]:
data['wordnet'][0]

{'pos_sent': 5.125, 'neg_sent': 1.375, 'pos_count': 13, 'neg_count': 1}

In [105]:
data['N_negword'] = [vader['neg_count'] for vader in data['wordnet']]
data['N_posword'] = [vader['pos_count'] for vader in data['wordnet']]
data['w_possent'] = [vader['pos_sent'] for vader in data['wordnet']]
data['w_negsent'] = [vader['neg_sent'] for vader in data['wordnet']]

In [72]:
def check(num):
    print('1.원문 : ', data['s_review'][num])
    print('2.번역 : ', data['번역'][num])
    print('3.긍부정단어수 : ', swn_polarity(data['번역'][num]))
    print('4.VADER : ',data['vader'][num])

In [101]:
check(6787)

1.원문 :  6시간 중에 1시간은 맥주의 재료와 종류 등에 대해 간단히 알려주시고 남은 5시간은 정말 딱 맞춰서 맥주를 만들었어요 당화시키고 홉 넣고 끓이고 식히는 중간 중간에 아이홉 공방에서 판매 중인 맥주들에 대해 하나 하나 그 스타일과 간단한 히스토리 등에 대해 알려주시고 자칫 지루할 수 있는 기 다림의 시간을 빙고게임과 함께 하니 후딱 지나갔네요 다음에 도 또 개인적으로 방문하고 싶은 곳입니다
2.번역 :  One hour of six hours is a brief reminder of the ingredients and types of beer, and the remaining five hours are really just right, making beer. One by one, tell me about the style and simple history of the beer sold in the I-hop workshop in the middle of saccharification, hop, boiling and cooling.
3.긍부정단어수 :  {'pos_sent': 1.25, 'neg_sent': 0.625, 'pos_count': 3, 'neg_count': 1}
4.VADER :  {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


## 분석결과 

- VADER가 Wordnet보다 성능이 우수한 것으로 판단된다. 
- Wordnet에서 첫번째 단어 뜻을 가져와서 쓰는점이 성능을 저하 시킬 것으로 보인다.
- VADER는 집단지성을 기반으로 만들었기 때문에, 단순 긍/부정 단어의 수를 새는 것보다 좋은 성능을 보이는 것으로 생각된다. 

## 단어수 체크

In [121]:
data['review_length'] = data['s_review'].apply(len)

In [127]:
# 데이터 저장 
data.to_csv('/Users/jin/Documents/pythonstudy/NLP/frip_data_result/frip_data_result.csv')